In [2]:
!pip install pyTelegramBotAPI
# !pip install torch
# !git clone https://github.com/huggingface/diffusers
# !pip install diffusers

In [8]:
import telebot
import os
from telebot import types
from telebot.types import Message
import threading
import queue
# добавления файла с токеном
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/cv_bot/secret.py' '/content'
from secret import secrets
# !mkdir '/content/lora1.4'
!git clone https://github.com/jilly09/cv-krosh.git
!mkdir '/content/lora'
!cp '/content/cv-krosh/lora/pytorch_lora_weights_new14_final.safetensors' '/content/lora'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'cv-krosh' already exists and is not an empty directory.
mkdir: cannot create directory ‘/content/lora’: File exists


In [ ]:
!cp '/content/drive/MyDrive/cv_bot/pytorch_lora_weights.safetensors' '/content/lora1.4'

In [11]:
from diffusers import AutoPipelineForImage2Image
from diffusers import AutoPipelineForText2Image
from diffusers import StableDiffusionPipeline
from diffusers.utils import load_image, make_image_grid
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")
pipeline_text2image.load_lora_weights("lora")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
# архив с разрешениями пользователей
res_width = {

}
res_height = {

}

In [24]:
token = secrets.get('BOT_API_TOKEN')
bot = telebot.TeleBot(token)

# start
@bot.message_handler(commands=['start'])
def start_message(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    start_button = types.KeyboardButton("Информация о боте")
    help_button = types.KeyboardButton("Информация о командах")
    markup.add(start_button, help_button)
    bot.send_message(message.chat.id, text='Привет, {0.first_name} 👋. Это бот для генерации изображений с Крошем 🐰 из мультсериала "Смешарики".'.format(message.from_user), reply_markup=markup)
    bot.send_message(message.chat.id, text = """
список команд:
/start
`/text2image текст запроса`
`/resolution длинна и ширина`
`/image2image текста запроса`, изображение
         """.format(message.from_user), reply_markup=markup, parse_mode="Markdown")
    bot.send_message(message.chat.id, text='Для генерации изображения необходимо делать запрос на английском языке. Воспользуйся кнопками'.format(message.from_user), reply_markup=markup)
    user_name = message.from_user.id
    global res_width
    res_width[user_name] = 512
    global res_height
    res_height[user_name] = 512

# set resolution
@bot.message_handler(commands=['resolution'])
def set_resolution(message):
    try:
      res = message.text.split(' ')
      user_name = message.from_user.id
      if len(res) == 3:
        width = res[1]
        height = res[2]
        global res_width
        res_width[user_name] = int(width)
        global res_height
        res_height[user_name] = int(height)
        bot.send_message(message.chat.id, text="Разрешение применено успешно.")
      else:
        bot.send_message(message.chat.id, text="❌ Пожалуйста, укажите разрешение, например: `/resolution 1280 720`", parse_mode="Markdown")
    except ValueError:
      bot.send_message(message.chat.id, text="❌ Пожалуйста, укажите разрешение в числах, например: `/resolution 1280 720`", parse_mode="Markdown")
request_queue = queue.Queue()
processing_lock = threading.Lock()
# text2image
def process_queue():
    """Функция для обработки очереди"""
    while True:
        chat_id, prompt = request_queue.get()
        with processing_lock:
            try:
                bot.send_message(chat_id, f"🔮 Генерирую изображение по запросу: *{prompt}*", parse_mode="Markdown")
                user_name = chat_id
                widths = res_width[user_name]
                heights = res_height[user_name]
                img = pipeline_text2image(
                prompt = prompt,
                # num_inference_steps=30,
                #guidance_scale=7.5,
                width=widths,
                height=heights,
                ).images[0]
                bot.send_photo(chat_id, img, caption="✅ Ваше изображение готово!")
            except ValueError:
                bot.send_message(chat_id, f"❌ Ошибка. Пожалуйста измените разрешение изображения.")
            except RuntimeError:
                bot.send_message(chat_id, f"🚫 Ошибка при обработке: бот перегружен. Пожалуйста подождите.")
            except Exception as e:
                bot.send_message(chat_id, f"❌ Ошибка: {e}")
        request_queue.task_done()
worker_thread = threading.Thread(target=process_queue, daemon=True)
worker_thread.start()


@bot.message_handler(commands=["text2image"])
def handle_txt2img(message: Message):
    prompt = message.text.replace("/text2image", "").strip()
    if not prompt:
        bot.reply_to(message, "⚠️ Укажите текст запроса, например: `/text2image Krosh with chainsaw`", parse_mode="Markdown")
        return
    request_queue.put((message.chat.id, prompt))
    bot.reply_to(message, f"🔄 Ваш запрос *{prompt}* Обрабатывается. Ожидайте...", parse_mode="Markdown")


# image2image
task_queue2 = queue.Queue()
pending_requests2 = {}
lock = threading.Lock()

def worker():
    """Поток, обрабатывающий очередь задач."""
    while True:
        task = task_queue2.get()
        user_id, user_text, user_image = task

        try:
            bot.send_message(user_id, f"🔮 *Начата обработка:*\nЗапрос: `{user_text}`", parse_mode="Markdown")
            prev_image = load_image(user_image)
            pipeline_image2image = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to("cuda")
            prompt = user_text
            image = pipeline_image2image(prompt, image=prev_image, strength=0.8, guidance_scale=10.5).images[0]
            make_image_grid([prev_image, image], rows=1, cols=2)
            bot.send_photo(user_id, image)
            bot.send_message(user_id, f"✅ *Готово!*\nЗапрос: `{user_text}`\nРезультат: успешно обработан.", parse_mode="Markdown")
            image
        except RuntimeError:
            bot.send_message(user_id, f"🚫 Ошибка при обработке: бот перегружен. Пожалуйста подождите.")
        except Exception as e:
            bot.send_message(user_id, f"🚫 Ошибка при обработке: {e}")

        task_queue2.task_done()

threading.Thread(target=worker, daemon=True).start()

@bot.message_handler(commands=['image2image'])
def handle_image2image(message):
    try:
        user_id = message.chat.id
        user_text = message.text.split('/image2image', 1)[1].strip()

        if not user_text:
            bot.reply_to(message, "❌ Укажите запрос: `/image2image описание`", parse_mode="Markdown")
            return

        with lock:
            pending_requests2[user_id] = {'user_text': user_text}

        bot.reply_to(message, f"🖼 Отправьте **одно изображение** для запроса:\n*{user_text}*", parse_mode="Markdown")

    except Exception as e:
        bot.reply_to(message, f"🚫 Ошибка: {e}")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    try:
        user_id = message.chat.id
        with lock:
            if user_id not in pending_requests2:
                bot.reply_to(message, "⚠ Сначала введите `/image2image запрос`", parse_mode="Markdown")
                return

            if 'user_image' in pending_requests2[user_id]:
                bot.reply_to(message, "ℹ Изображение уже принято. Дождитесь обработки.")
                return
            file_id = message.photo[-1].file_id
            file_info = bot.get_file(file_id)
            file_url = f"https://api.telegram.org/file/bot{token}/{file_info.file_path}"
            user_text = pending_requests2[user_id]['user_text']
            user_image = file_url
            task_queue2.put((user_id, user_text, user_image))
            bot.reply_to(message, f"📥 Запрос обрабатывается.")
            del pending_requests2[user_id]

    except Exception as e:
        bot.reply_to(message, f"🚫 Ошибка: {e}")

@bot.message_handler(content_types=['text'])
def buttons(message):
    if (message.text == "Информация о боте"):
        bot.send_message(message.chat.id, text='Данный бот генерирует любые картинки с Крошем из мультфильма "Смешарики". Использовалась модель нейросети Stable Diffusion, которая была дообучена для генерации изображений с Крошем.')

    elif (message.text == "Информация о командах"):
         bot.send_message(message.chat.id, text="""
/start - запускает бота.
/text2image текст запроса - команда которая принимает запрос на английском языке от пользователя для генерации изображения. Например: `/text2image Krosh with chainsaw`.
/resolution длинна и ширина - команда которая задаёт собственные параметры разрешения изображения(по умолчанию стоит 512 на 512). Значения задаются через пробел, например: `/resolution 1600 900`.
/image2image текста запроса, изображение - комманда которая принимает текст запроса и следующим сообщение изображение которое надо изменить. Например: `/image2image add Krosh to this picture`.
         """, parse_mode="Markdown")

    else:
      bot.send_message(message.chat.id, text="Нет такой команды")


In [25]:
bot.polling(none_stop=True, interval=0)